In [ ]:
%matplotlib inline

import numpy as np
#np.set_printoptions(threshold=np.nan)
from matplotlib import pyplot as plt

import ctapipe
from ctapipe.io.hessio import hessio_event_source
import pyhessio

import sys
sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline")

import datapipe
import datapipe.denoising.wavelets_mrfilter as mrfilter

sys.path.append("/Users/jdecock/git/pub/jdhp-sap/sap-cta-data-pipeline/utils")
import common_functions as common

from datapipe.io import images
from datapipe.io import geometry_converter
import datapipe.io.montecarlo_calibration_astri as mc_calibration

In [ ]:
tel_id = 18
source = hessio_event_source("/Users/jdecock/data/astri_mini_array/proton/run10001.simtel.gz", allowed_tels=[tel_id])

In [ ]:
for ev in source:
    if tel_id in ev.dl0.tel:
        print(ev.dl0.tel[tel_id+1].adc_sums[1])

In [ ]:
#print(ev)

In [ ]:
# Pedestal
print(ev.mc.tel[tel_id].pedestal[0])
print(ev.mc.tel[tel_id].pedestal[1])

# Gain
print(ev.mc.tel[tel_id].dc_to_pe[0])
print(ev.mc.tel[tel_id].dc_to_pe[1])

# ADC sums
print(ev.r0.tel[tel_id].adc_sums[0])
print(ev.r0.tel[tel_id].adc_sums[1])

print(ev.mc.tel[tel_id].photo_electron_image)

ev.mc.tel[tel_id].reference_pulse_shape.shape

plt.plot(ev.mc.tel[tel_id].reference_pulse_shape[0])
plt.plot(ev.mc.tel[tel_id].reference_pulse_shape[1])

# TODO!!!
#help(pyhessio)
#pedestal, gains = get_mc_calibration_data(tel_id)
#print("pedestal:", pedestal)
#print("gains:", gains)

In [ ]:
# GET IMAGES ##############################################

pe_image = ev.mc.tel[tel_id].photo_electron_image   # 1D np array
uncalibrated_image = ev.r0.tel[tel_id].adc_sums     # ctapipe 0.4.0
pedestal = ev.mc.tel[tel_id].pedestal
gain = ev.mc.tel[tel_id].dc_to_pe
pixel_pos = ev.inst.pixel_pos[tel_id]

# calibrating
calibrated_image = mc_calibration.apply_mc_calibration(uncalibrated_image, pedestal, gain)

In [ ]:
# %load ~/git/pub/jdhp-sap/snippets/ctapipe/plot_pixels_index.py
#!/usr/bin/env python3

"""
Display the pixel layout of the givent telescope as defined in a given simtel file.
"""

(pos_x_list, pos_y_list) = pixel_pos
pos_x_list = [float(pos.value) for pos in pos_x_list]
pos_y_list = [float(pos.value) for pos in pos_y_list]

assert len(pos_x_list) == len(pos_y_list)

# PLOT ####################################################################

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 12))

# Scatter method ##################

ax.scatter(pos_x_list,    # x
           pos_y_list,    # y
           s=0.2,         # radius
           c="black",     # color
           alpha=0.75)

ax.plot(pos_x_list,    # x
        pos_y_list,    # y
        "k-",
        alpha=0.25)

for pixel_index in range(len(pos_x_list)):
    ax.text(pos_x_list[pixel_index], pos_y_list[pixel_index], str(pixel_index), fontsize=8)

ax.set_title("Pixels position (telescope {})".format(tel_id), fontsize=16)

ax.set_xlabel("x position (m)", fontsize=16)
ax.set_ylabel("y position (m)", fontsize=16)

ax.set_xlim(-0.21, 0.21)
ax.set_ylim(-0.21, 0.21)

plt.show()

### Crop image

In [ ]:
# cropping ADC image
cropped_adc_sums = geometry_converter.astry_to_3d_array(uncalibrated_image, crop=True)

# cropping PE image
cropped_pe_img = geometry_converter.astry_to_2d_array(pe_image, crop=True)

# cropping calibrated image
cropped_img = geometry_converter.astry_to_2d_array(calibrated_image, crop=True)

# cropping pedestal and gain
cropped_pedestal = geometry_converter.astry_to_3d_array(pedestal, crop=True)
cropped_gains = geometry_converter.astry_to_3d_array(gain, crop=True)

# cropping pixel positions
cropped_pixel_pos = geometry_converter.astry_to_3d_array(pixel_pos, crop=True)

In [ ]:
images.plot(cropped_adc_sums[0])

In [ ]:
images.plot(cropped_adc_sums[1])

In [ ]:
images.plot(cropped_pe_img)

In [ ]:
images.plot(cropped_img)

In [ ]:
images.plot(cropped_pedestal[0])

In [ ]:
images.plot(cropped_pedestal[1])

In [ ]:
images.plot(cropped_gains[0])

In [ ]:
images.plot(cropped_gains[1])

In [ ]:
images.plot(cropped_pixel_pos[0])

In [ ]:
images.plot(cropped_pixel_pos[1])

### Don't crop image

In [ ]:
# cropping ADC image
not_cropped_adc_sums = geometry_converter.astry_to_3d_array(uncalibrated_image, crop=False)

# cropping PE image
not_cropped_pe_img = geometry_converter.astry_to_2d_array(pe_image, crop=False)

# cropping calibrated image
not_cropped_img = geometry_converter.astry_to_2d_array(calibrated_image, crop=False)

# cropping pedestal and gain
not_cropped_pedestal = geometry_converter.astry_to_3d_array(pedestal, crop=False)
not_cropped_gains = geometry_converter.astry_to_3d_array(gain, crop=False)

# cropping pixel positions
not_cropped_pixel_pos = geometry_converter.astry_to_3d_array(pixel_pos, crop=False)

In [ ]:
images.plot(not_cropped_adc_sums[0])

In [ ]:
images.plot(not_cropped_adc_sums[1])

In [ ]:
images.plot(not_cropped_pe_img)

In [ ]:
images.plot(not_cropped_img)

In [ ]:
images.plot(not_cropped_pedestal[0])

In [ ]:
images.plot(not_cropped_pedestal[1])

In [ ]:
images.plot(not_cropped_gains[0])

In [ ]:
images.plot(not_cropped_gains[1])

In [ ]:
images.plot(not_cropped_pixel_pos[0])

In [ ]:
images.plot(not_cropped_pixel_pos[1])

## Safety check

In [ ]:
np.all(not_cropped_adc_sums[0][8:6*8,8:6*8] == cropped_adc_sums[0])

In [ ]:
np.all(not_cropped_adc_sums[1][8:6*8,8:6*8] == cropped_adc_sums[1])

In [ ]:
np.all(not_cropped_pe_img[8:6*8,8:6*8] == cropped_pe_img)

In [ ]:
np.all(not_cropped_img[8:6*8,8:6*8] == cropped_img)

In [ ]:
np.all(not_cropped_pedestal[0][8:6*8,8:6*8] == cropped_pedestal[0])

In [ ]:
np.all(not_cropped_pedestal[1][8:6*8,8:6*8] == cropped_pedestal[1])

In [ ]:
np.all(not_cropped_gains[0][8:6*8,8:6*8] == cropped_gains[0])

In [ ]:
np.all(not_cropped_gains[1][8:6*8,8:6*8] == cropped_gains[1])

In [ ]:
np.all(not_cropped_pixel_pos[0][8:6*8,8:6*8] == cropped_pixel_pos[0])

In [ ]:
np.all(not_cropped_pixel_pos[1][8:6*8,8:6*8] == cropped_pixel_pos[1])